In [1]:
import requests as rq
import json
import pandas as pd
pd.set_option('display.precision', 4,
              'display.colheader_justify', 'center')
import numpy as np
import warnings
import pytz
import datetime
import time
from IPython.display import clear_output

In [2]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

In [4]:
use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_pro_key()
}

In [5]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")    

In [6]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

## Get list of networks

In [7]:
networks_list_response = get_response("/onchain/networks",
                                      use_pro, 
                                      "",
                                      PRO_URL)

In [17]:
networks_list_response["data"][0]

{'id': 'eth',
 'type': 'network',
 'attributes': {'name': 'Ethereum', 'coingecko_asset_platform_id': 'ethereum'}}

In [13]:
df_net = pd.DataFrame(networks_list_response["data"])

## Get DEX for a specific network

In [18]:
network = "eth"

dex_list_response = get_response(f"/onchain/networks/{network}/dexes",
                                 use_pro, 
                                 "",
                                 PRO_URL)

In [84]:
df_dex = pd.DataFrame(dex_list_response["data"])

## Get trending pools for a specific network

In [74]:
network = "eth"
dex = "uniswap_v2"

url_dict = {
    "trending_pools": f"/onchain/networks/{network}/trending_pools",
    "top_pools": f"/onchain/networks/{network}/pools",
    "top_pools_dex": f"/onchain/networks/{network}/dexes/{dex}/pools"
}

In [69]:
def collect_response(list_response):    

    response_all = []

    for response in list_response["data"]:
        all_attributes = response["attributes"]
        daily_tx = all_attributes["transactions"]["h24"]
        rel = response["relationships"]
        
        temp_dict = dict(
            pair = all_attributes["name"],
            dex = rel["dex"]["data"]["id"],
            add = all_attributes["address"],
            fdv_usd = all_attributes["fdv_usd"],
            market_cap_usd = all_attributes["market_cap_usd"],
            daily_volume = all_attributes["volume_usd"]["h24"],
            daily_price_change = all_attributes["price_change_percentage"]["h24"],
            daily_buys = daily_tx["buys"],
            daily_sells = daily_tx["sells"],
            daily_buyers = daily_tx["buyers"],
            daily_sellers = daily_tx["sellers"]
        )
        
        response_all.append(temp_dict)

    return response_all

In [75]:
def get_trending_pools(sort_by_col):

    trendpool_list_response = get_response(url_dict["trending_pools"],
                                           use_pro, 
                                           "",
                                           PRO_URL)
    
    trendpool_all = collect_response(trendpool_list_response)    

    return pd.DataFrame(trendpool_all).sort_values(by = [f"{sort_by_col}"],
                                                   ascending = False)

In [77]:
#get_trending_pools("daily_volume")

## Get top pools for a specific network

In [81]:
def get_top_pools_network(sort_by_col):

    toppool_list_response = get_response(url_dict["top_pools"],
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [80]:
#get_top_pools_network("daily_volume")

## Get top pools for a specific DEX

In [82]:
def get_top_pools_dex(sort_by_col):

    toppool_list_response = get_response(url_dict["top_pools_dex"],
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [83]:
get_top_pools_dex("daily_volume")

,pair,dex,add,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers
12,WETH / USDT,uniswap_v2,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,7021499936,7012227857.28965,8752184.56988771,3.89,1677,1029,1058,780
2,SATO / WETH,uniswap_v2,0x1e79d6529f271876d202cbb216b856165d862353,8409906,None,8086545.06648121,-29.96,4251,3905,1362,1253
14,deer / WETH,uniswap_v2,0x09b97f6b36b3997612357cd4457b272ff09313c3,12465.62,None,758012.39785698,309.36,1353,1228,823,704
19,SON / WETH,uniswap_v2,0xa82ed4a046d97343487e3d1e0c1a994985037e75,12704.87,None,521639.224818656,-71.64,1170,911,607,534
17,CATE / WETH,uniswap_v2,0x68d66f784b49c2f3acf80e549cde65c81a0a1e12,24592510,24765651.4303498,3941325.80275021,-31.03,1249,1171,709,698
6,MUGI / WETH,uniswap_v2,0xb918f52f6b9763af818c256fd95fcb873072a016,2377195,None,3779411.2918456,93.64,2280,1747,1160,929
1,XMW / WETH,uniswap_v2,0x0da7096f14303eddd634c0241963c064e0244984,5847808,None,3413191.56572372,12481.97,6875,3294,1940,1436
4,MISHA / WETH,uniswap_v2,0xd87c1ff13e7bfd9c13132b6fcffd6b69e0e08b1f,606530,None,3388645.28378828,12031.56,2777,2111,1398,1036
3,Neiro / WETH,uniswap_v2,0xc555d55279023e732ccd32d812114caf5838fd46,386832300,385518695.06745,31596173.086123,14.45,3029,2725,1442,1366
7,MOODENG / WETH,uniswap_v2,0x470dc172d6502ac930b59322ece5345dd456a03d,2790364,None,2864670.2700903,-34.73,1838,1402,1002,812
